# Eigenvalue problems (Unit 2.2) 

We solve the generalized eigenvalue problem
$$A u=\lambda M u$$,

where A comes from $\int \nabla u \nabla v$, and $M$ from $\int uv$, on the space $H_0^1$

In [1]:
import netgen.gui
%gui tk
from netgen.geom2d import unit_square
from ngsolve import *
from math import pi
import scipy.linalg
from scipy import random

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))


We setup a stiffness matrix A and a mass matrix M, and declare a preconditioner for A:

In [2]:
fes = H1(mesh, order=4, dirichlet=[1,2,3,4])
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))
pre = Preconditioner(a, "multigrid")

m = BilinearForm(fes)
m += SymbolicBFI(u*v)

a.Assemble()
m.Assemble()

u = GridFunction(fes)

The inverse iteration is
$$u_{n+1}=A^{−1}Mu_n,$$

where the Rayleigh quotient
$$\rho_n=\frac{\langle Au_n,u_n \rangle}{\langle Mu_n,u_n\rangle}$$

converges to the smallest eigen-value, with rate of convergence $\lambda_1/\lambda_2$.

The preconditioned inverse iteration (PINVIT), see [Knyazef+Neymeyr], replaces $A^{−1}$
by an approximate inverse $C^{−1}$

$$\rho_n=\frac{\langle Au_n,u_n\rangle}{\langle Mu_n,u_n\rangle}$$

$$w_n=C^{−1}(Au_n−\rho Mu_n)$$
$$u_{n+1}=u_n+\alpha w_n$$

The optimal step-size $\alpha$ is found by minimizing the Rayleigh-quotient on a two-dimensional space:

$$u_{n+1}=\arg \min_{v\in \lbrace u_n,w_n\rbrace} \frac{\langle Av,v\rangle}{\langle Mv,v\rangle}$$

This minimization problem can be solved by a small eigenvalue problem
$$ay=\lambda my$$

with matrices
$$a = \left(
\begin{array}[cc]
\ \langle Au_n,u_n\rangle & \langle Au_n,w_n\rangle\\
\langle Aw_n,u_n\rangle & \langle Aw_n,w_n\rangle
\end{array}
\right) \hspace{3em} m = \left(
\begin{array}[cc]
\ \langle Mu_n,u_n\rangle & \langle Mu_n,w_n\rangle\\
\langle Mw_n,u_n\rangle & \langle Mw_n,w_n\rangle
\end{array}
\right) $$

Then, the new iterate is

$$u_{n+1}=y_1 u_n+y_2 w_n$$

where $y$ is the eigenvector corresponding to the smaller eigenvalue.
### Implementation in NGSolve

Create some help vectors:


In [3]:
r = u.vec.CreateVector()
w = u.vec.CreateVector()
Mu = u.vec.CreateVector()
Au = u.vec.CreateVector()
Mw = u.vec.CreateVector()
Aw = u.vec.CreateVector()


random initial condition, zero on the boundary

In [4]:
# Note how we can use numpy with NGSolve vectors!
# FV stands for FlatVector
r.FV().NumPy()[:] = random.rand(fes.ndof)
# Linear operator projecting to true/false bits of BitArray
# mask, depending on argument range
# Also has a Project() method to project a vector inline
u.vec.data = Projector(fes.FreeDofs(), range=True) * r

# much better than the old way...
#freedofs = fes.FreeDofs()    
#for i in range(len(u.vec)):
#    u.vec[i] = random() if freedofs[i] else 0   

We solve the small generalized eigenvalue problem using eigh:

Find eigenvalues w and optionally eigenvectors v of matrix `a`, where `a` and `b` are complex Hermitian or real symmetric matrices and `b` is positive definite


In [6]:
for i in range(20):
    Au.data = a.mat * u.vec
    Mu.data = m.mat * u.vec
    auu = InnerProduct(Au, u.vec)
    muu = InnerProduct(Mu, u.vec)
    # Rayleigh quotiont
    rho = auu/muu
    print (rho / (pi**2)) # I think for this problem, the eigenvalues turn out to be multiples of pi^2
    # residual
    r.data = Au - rho * Mu
    w.data = pre.mat * r.data
    w.data = 1/Norm(w) * w
    Aw.data = a.mat * w
    Mw.data = m.mat * w

    # setup and solve 2x2 small eigenvalue problem
    asmall = Matrix(2,2)
    asmall[0,0] = auu
    asmall[0,1] = asmall[1,0] = InnerProduct(Au, w)
    asmall[1,1] = InnerProduct(Aw, w)
    msmall = Matrix(2,2)
    msmall[0,0] = muu
    msmall[0,1] = msmall[1,0] = InnerProduct(Mu, w)
    msmall[1,1] = InnerProduct(Mw, w)
    # print ("asmall =", asmall, ", msmall = ", msmall)
    
    
    eval,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    # print (eval, evec)
    u.vec.data = float(evec[0,0]) * u.vec + float(evec[1,0]) * w
    
Draw (u)

27.972629879022797
2.0786451400176014
2.0011792784626827
2.0001619873036085
2.000034831232633
2.000008246631434
2.0000019953397645
2.0000004894520025
2.000000120869217
2.000000030054258
2.0000000075235227
2.0000000019135467
2.000000000511369
2.000000000160099
2.000000000071894
2.0000000000497096
2.0000000000441225
2.000000000042714
2.0000000000423577
2.000000000042268



### Simultaneous iteration for several eigenvalues

Declare GridFunction with multiple components to store several eigenvectors.


In [11]:
num = 10
u = GridFunction(fes, multidim=num)


Create list of help vectors:


In [12]:
r = u.vec.CreateVector()
Av = u.vec.CreateVector()
Mv = u.vec.CreateVector()

vecs = []
for i in range(2*num):
    vecs.append (u.vec.CreateVector())

for v in u.vecs:
    r.FV().NumPy()[:] = random.rand(fes.ndof)
    v.data = Projector(fes.FreeDofs(), True) * r

# old way
#freedofs = fes.FreeDofs()    
#for v in u.vecs:
#    for i in range(len(u.vec)):
#        v[i] = random() if freedofs[i] else 0



Compute $num$ residuals, and solve small eigenvalue problem on $2num$-dimensional space

In [13]:
asmall = Matrix(2*num, 2*num)
msmall = Matrix(2*num, 2*num)
rhos = num * [1]

for i in range(20):
    
    for j in range(num):
        vecs[j].data = u.vecs[j]
        r.data = a.mat * vecs[j] - rhos[j] * m.mat * vecs[j]
        vecs[num+j].data = pre.mat * r

    for j in range(2*num):
        Av.data = a.mat * vecs[j]
        Mv.data = m.mat * vecs[j]
        for k in range(2*num):
            asmall[j,k] = InnerProduct(Av, vecs[k])
            msmall[j,k] = InnerProduct(Mv, vecs[k])

    ev,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    rhos[:] = ev[0:num]
    print (i, ":", [rho/pi**2 for rho in rhos])
    
    for j in range(num):
        u.vecs[j][:] = 0.0
        for k in range(2*num):
            u.vecs[j].data += float(evec[k,j]) * vecs[k]
            
Draw (u)

0 : [9.707319064124958, 61.384253117110774, 66.17820958237785, 69.04094119858487, 84.73605522656003, 89.34325969621554, 91.68839867830556, 98.80839508826868, 109.91801400369201, 124.77011533297116]
1 : [2.024324236790256, 5.458807306602363, 6.286163725215133, 10.738747007591671, 12.031042228536256, 14.91354228688427, 17.876052005882798, 20.659543931381272, 23.263073421993237, 35.535499988259275]
2 : [2.001200106455104, 5.006450901113249, 5.028597732937264, 8.119732597601683, 10.111026822994479, 10.594547833277227, 13.163464205654073, 15.629715905774423, 18.390101604114673, 22.226228751010957]
3 : [2.0002337026485892, 5.000503372971397, 5.001315769359331, 8.01048048456922, 10.009996680722088, 10.062674363097718, 13.033541471545037, 13.88180225549825, 17.39372785742104, 20.25243730229141]
4 : [2.0000532064104313, 5.000087741811666, 5.000161928051458, 8.001118517518545, 10.001308177619594, 10.008423165846647, 13.00829666250324, 13.295113976507041, 17.131828952784893, 19.514856337170354]
5

The multidim-component select in the Visualization dialog box allows to switch between the components of the multidim-GridFunction.